# Track demo

Track vessel by mmsi. Note if querying for large areas then you should consider using a larger notebook or breaking the area into smaller queries and using [user defined functions](https://docs.tiledb.com/cloud/client-api/serverless-array-udfs)

In [ ]:
import time

import datashader as ds
from datashader.utils import lnglat_to_meters
import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews.element import tiles
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import geopandas as gpd
import tiledb

import warnings
warnings.filterwarnings("ignore")

hv.extension("bokeh", "matplotlib")

tiledb_mmsi_uri = 'tiledb://spire-data/mmsi_ais'

config = tiledb.Config()

# Set value
config["sm.memory_budget"] = 50_000_000
config["sm.memory_budget_var"] = 50_000_000

ctx = tiledb.Ctx(config)

The mmsi identifier we will use is `308371000`

In [ ]:
%%time

t1 = np.datetime64('2019-08-01T00:00:00')
t2 = np.datetime64('2019-08-15T00:00:00')

with tiledb.SparseArray(tiledb_mmsi_uri, ctx=ctx) as arr:
    df = arr.query(attrs=['latitude', 'longitude'], dims=["ts_pos_utc", "mmsi"]).df[t1:t2, 308371000]

print(f"Retrieved {len(df['longitude'])} points")
    
df.sort_values(by=['ts_pos_utc'], inplace=True, ascending=True)
df.loc[:, 'x'], df.loc[:, 'y'] = lnglat_to_meters(df.longitude,df.latitude)
df

Using existing data science tooling we can plot these points on a map

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='epsg:4326')
ax = gdf.plot(markersize=1.5, figsize=(8, 8))
plt.autoscale(False)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.to_crs(gdf.crs).plot(ax=ax, color='none', edgecolor='black')

In [ ]:
%%time

bkgrd = tiles.EsriImagery().opts(xaxis=None, yaxis=None, width=700, height=500)
opts = hv.opts.RGB(width=500, height=500)
bkgrd * hd.datashade(hv.Path(gdf, kdims=['x','y']), cmap='red', normalization='linear', aggregator=ds.any()).opts(opts)